# MCMCでマルチレベル分析　HLM

In [2]:
library(psych)
library(dplyr)

In [2]:
dat<-read.csv("hlm_data.csv")
dat %>% head(3)

順列,個人成績,集団成績,条件,誤答率,平均反応時間
1,9,10,1,0.0,517.8
1,8,10,1,0.0,591.1
1,10,10,1,0.1,484.0


In [3]:
colnames(dat)<- c("P", "ind_per","group_per", "cond", "error", "RT")
dat %>% head(3)

P,ind_per,group_per,cond,error,RT
1,9,10,1,0.0,517.8
1,8,10,1,0.0,591.1
1,10,10,1,0.1,484.0


In [4]:
#install.packages("ICC")
library(ICC)

In [5]:
# 級内相関
ICCest(x = factor(P), y = error, dat)

$ICC
[1] 0.9638784

$LowerCI
[1] 0.8852431

$UpperCI
[1] 0.9941895

$N
[1] 6

$k
[1] 4

$varw
[1] 0.0008111111

$vara
[1] 0.02164389

In [6]:
# データを中心化
dat$error_m <- tapply(dat$error, dat$P, mean, na.rm = TRUE)[as.character(dat$P)]

In [7]:
dat$error_g <- dat$error - dat$error_m

In [8]:
dat$group_per <- dat$group_per - mean(dat$group_per, na.rm = T)

In [9]:
head(dat)

P,ind_per,group_per,cond,error,RT,error_m,error_g
1,9,3.666667,1,0.00,517.8,0.0375,-0.0375
1,8,3.666667,1,0.00,591.1,0.0375,-0.0375
1,10,3.666667,1,0.10,484.0,0.0375,0.0625
1,9,3.666667,1,0.05,469.9,0.0375,0.0125
2,3,-2.333333,0,0.30,456.8,0.2750,0.0250
2,2,-2.333333,0,0.28,477.8,0.2750,0.0050


In [12]:
# 素直に線形推定
library(lmerTest)

In [13]:
result_ml<- lmer(ind_per ~ (1+error_g|P) + error_g + group_per, dat, REML = F)

In [14]:
summary(result_ml)

Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: ind_per ~ (1 + error_g | P) + error_g + group_per
   Data: dat

     AIC      BIC   logLik deviance df.resid 
    76.9     85.2    -31.5     62.9       17 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.56341 -0.74832  0.01909  0.56946  1.76386 

Random effects:
 Groups   Name        Variance  Std.Dev. Corr 
 P        (Intercept)   0.06831  0.2614       
          error_g     145.50825 12.0627  -1.00
 Residual               0.69028  0.8308       
Number of obs: 24, groups:  P, 6

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)  5.70833    0.20037  7.50086  28.489 6.42e-09 ***
error_g     -1.47894    9.11494  3.88389  -0.162    0.879    
group_per    1.11246    0.06982 11.22585  15.934 4.65e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
          (Intr) errr

In [15]:
# 目的変数って正規性を仮定していいの？
shapiro.test(dat$ind_per)


	Shapiro-Wilk normality test

data:  dat$ind_per
W = 0.89711, p-value = 0.0187


# 以下Rstanの迷宮

In [20]:
# 最尤法でHLMを行うのに必要
#install.packages("lmerTest")
# GitHub状のパッケージをインストールするためのパッケージ
#install.packages("devtools")
# Rstan
#install.packages("rstan",repos='https://cloud.r-project.org/', dependencies=TRUE)

Installing package into 'C:/Users/mimus/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'lmerTest' is in use and will not be installed"Installing package into 'C:/Users/mimus/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'devtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mimus\AppData\Local\Temp\RtmpmG3YoN\downloaded_packages


Installing package into 'C:/Users/mimus/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'rstan' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mimus\AppData\Local\Temp\RtmpmG3YoN\downloaded_packages


In [21]:
pkgbuild::has_build_tools(debug = TRUE)

Scanning R CMD config CC...
cc_path: gcc 
install_path: C:/Users/mimus/ANACON~2/Library 
Found compatible gcc from R CMD config CC


[1] TRUE

In [23]:
dotR <- file.path(Sys.getenv("HOME"), ".R")
if (!file.exists(dotR)) dir.create(dotR)
M <- file.path(dotR, ifelse(.Platform$OS.type == "windows", "Makevars.win", "Makevars"))
if (!file.exists(M)) file.create(M)
cat("\nCXX14FLAGS=-O3 -march=native -mtune=native",
    if( grepl("^darwin", R.version$os)) "CXX14FLAGS += -arch x86_64 -ftemplate-depth-256" else
    if (.Platform$OS.type == "windows") "CXX11FLAGS=-O3 -march=corei7 -mtune=corei7" else
    "CXX14FLAGS += -fPIC",
    file = M, sep = "\n", append = TRUE)

#### 下準備終わり！

# よっしゃ！Rstan使うで！

In [11]:
# Rstanの呼び出し
library(rstan)

# 清水先生作のglmmstanを使う。
library(doParallel)
source("http://bit.ly/glmmstan")
source("glmmstan.R")

In [18]:
# 並列で推定計算を実行させる
options(mc.cores = parallel::detectCores())
# コンパイル後のStanプログラムをハードディスクに保存する。（人間語を常にコンピュータ後に代えてくれる）
rstan_options(auto_write = TRUE)
# Windowsは実行時間を向上できる
Sys.setenv(LOCAL_CPPFLAGS = '-march=corei7')

In [19]:
result_mcmc<- glmmstan(ind_per ~ group_per+ (1+error_g|P), dat)


Compiling the stan code.
DIAGNOSTIC(S) FROM PARSER:
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: Deprecated function 'normal_log'; please replace suffix '_log' with '_lpdf' for density functions or '_lpmf' for mass functions
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in the Stan language; use = instead.
Info: assignment operator <- deprecated in 

ERROR: Error:  サイズ 10.9 Gb のベクトルを割り当てることができません 


Σ（・□・；）ふぁっ

え、じゃあ、おとなしく「一般的」と呼ばれてる方法で行くか...

In [10]:
library(glmmML)

In [12]:
lm<-glmmML(ind_per ~ group_per + error_g , data = dat, family = poisson, cluster = P)

In [13]:
summary(lm)


Call:  glmmML(formula = ind_per ~ group_per + error_g, family = poisson,      data = dat, cluster = P) 


              coef se(coef)       z Pr(>|z|)
(Intercept) 1.5809  0.10020 15.7779 0.00e+00
group_per   0.2208  0.03807  5.7991 6.67e-09
error_g     0.7129  3.12797  0.2279 8.20e-01

Scale parameter in mixing distribution:  3.961e-07 gaussian 
Std. Error:                              0.5904 

        LR p-value for H_0: sigma = 0:  0.5 

Residual deviance: 14.86 on 20 degrees of freedom 	AIC: 22.86 


ランダム効果も何もあったもんじゃないな (' へ')ノ